In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\python312\lib\site-packages\vboxapi-1.0-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from sklearn.datasets import make_classification

In [5]:
## create the dataset
X,y= make_classification(n_samples=1000,n_features=10,n_classes=2,random_state=42)

In [6]:
pd.DataFrame(X)

,0,1,2,3,4,5,6,7,8,9
0,0.964799,-0.066449,0.986768,-0.358079,0.997266,1.181890,-1.615679,-1.210161,-0.628077,1.227274
1,-0.916511,-0.566395,-1.008614,0.831617,-1.176962,1.820544,1.752375,-0.984534,0.363896,0.209470
2,-0.109484,-0.432774,-0.457649,0.793818,-0.268646,-1.836360,1.239086,-0.246383,-1.058145,-0.297376
3,1.750412,2.023606,1.688159,0.006800,-1.607661,0.184741,-2.619427,-0.357445,-1.473127,-0.190039
4,-0.224726,-0.711303,-0.220778,0.117124,1.536061,0.597538,0.348645,-0.939156,0.175915,0.236224
...,...,...,...,...,...,...,...,...,...,...
995,-1.367638,1.462255,-1.154918,-0.290454,-0.413424,0.032396,1.545490,1.428760,1.687092,1.072542
996,-1.514876,-3.221016,-1.300744,0.395599,-0.527994,1.353069,1.777506,-1.680870,1.798510,0.034272
997,1.674633,1.754933,1.586154,0.018402,-1.514470,0.321593,-2.417694,0.692723,-1.503850,0.225264
998,-0.778609,-0.835689,-0.194842,1.097220,0.180071,-0.272933,-0.533188,-0.497354,2.472138,0.867187


In [7]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test  = train_test_split(X,y,test_size=0.30,random_state=42)

In [8]:
#Model training
from sklearn.linear_model import LogisticRegression
logistic = LogisticRegression()

In [9]:
logistic.fit(X_train,y_train)

LogisticRegression()

In [10]:
y_pred = logistic.predict(X_test)
print(y_pred)

[0 1 0 1 0 1 0 0 0 0 0 1 0 1 0 0 1 1 1 1 0 0 1 0 0 1 0 0 1 1 1 1 0 0 0 0 1
 1 1 1 0 1 1 0 0 0 1 1 1 1 0 1 0 0 1 0 1 0 1 0 1 0 0 1 1 1 0 0 1 1 1 1 1 0
 1 0 0 1 0 1 0 0 1 0 1 0 0 0 0 1 1 1 1 1 1 1 0 0 1 0 1 0 1 0 0 1 0 1 1 1 1
 1 1 1 1 0 0 1 0 0 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 0 0 0 1 1 0 0 0 0 0 1 0
 0 0 1 0 0 1 0 0 1 0 1 1 0 1 0 0 0 0 0 0 0 1 1 0 0 0 1 1 0 0 1 1 1 1 1 1 1
 0 0 0 0 1 0 0 0 0 1 0 0 1 1 1 0 1 0 0 0 1 1 1 1 1 0 0 0 0 0 1 0 1 0 1 1 0
 0 1 1 1 0 1 1 0 0 0 0 0 0 0 0 0 1 1 0 1 0 1 1 0 1 0 1 0 1 1 1 0 0 1 1 1 1
 0 1 0 1 1 0 0 0 1 1 0 1 1 0 0 1 0 0 0 0 1 1 0 1 0 1 1 1 0 0 1 0 1 1 0 1 1
 1 1 1 0]


In [11]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

In [12]:
score = accuracy_score(y_test,y_pred)
print(score)
cm = confusion_matrix(y_test,y_pred)
print(cm)
print(classification_report(y_test,y_pred))

0.8466666666666667
[[118  17]
 [ 29 136]]
              precision    recall  f1-score   support

           0       0.80      0.87      0.84       135
           1       0.89      0.82      0.86       165

    accuracy                           0.85       300
   macro avg       0.85      0.85      0.85       300
weighted avg       0.85      0.85      0.85       300



### Hyperparameter Tuning and Cross Validation

## GridSearch CV


In [13]:
model =LogisticRegression()
penalty = ['l1','l2','elasticnet']
c_values = [100,10,1,0.1,0.01]
solver = ['newton-cg','lbfgs','liblinear','sag','saga']

In [21]:

# Build compatible parameter combinations
params=[]
for p in penalty:
    for s in solver:
        if (p == 'l1' and s not in ['liblinear', 'saga']):
            continue
        if (p == 'elasticnet' and s != 'saga'):
            continue
        if (p == 'elasticnet'):
            params.append({'penalty': [p], 'C': c_values, 'solver': [s], 'l1_ratio': [0.5]})
        else:
            params.append({'penalty': [p], 'C': c_values, 'solver': [s]})

In [22]:
## GridSearch CV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
cv = StratifiedKFold()
grid = GridSearchCV(estimator=model,param_grid=params,scoring='accuracy',cv=cv,n_jobs=-1)

In [23]:
grid

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             estimator=LogisticRegression(), n_jobs=-1,
             param_grid=[{'C': [100, 10, 1, 0.1, 0.01], 'penalty': ['l1'],
                          'solver': ['liblinear']},
                         {'C': [100, 10, 1, 0.1, 0.01], 'penalty': ['l1'],
                          'solver': ['saga']},
                         {'C': [100, 10, 1, 0.1, 0.01], 'penalty': ['l2'],
                          'solver': ['newton-cg']},
                         {'C': [100, 10, 1, 0.1, 0.01], 'penalty': ['l2'],
                          'solver': ['lbfgs']},
                         {'C': [100, 10, 1, 0.1, 0.01], 'penalty': ['l2'],
                          'solver': ['liblinear']},
                         {'C': [100, 10, 1, 0.1, 0.01], 'penalty': ['l2'],
                          'solver': ['sag']},
                         {'C': [100, 10, 1, 0.1, 0.01], 'penalty': ['l2'],
                          'solver': ['saga']},
                         {'C': [100, 10, 1, 0.1, 0.01], 'l1_ratio': [0.5],
                          'penalty': ['elasticnet'], 'solver': ['saga']}],
             scoring='accuracy')

In [24]:
grid.fit(X_train,y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             estimator=LogisticRegression(), n_jobs=-1,
             param_grid=[{'C': [100, 10, 1, 0.1, 0.01], 'penalty': ['l1'],
                          'solver': ['liblinear']},
                         {'C': [100, 10, 1, 0.1, 0.01], 'penalty': ['l1'],
                          'solver': ['saga']},
                         {'C': [100, 10, 1, 0.1, 0.01], 'penalty': ['l2'],
                          'solver': ['newton-cg']},
                         {'C': [100, 10, 1, 0.1, 0.01], 'penalty': ['l2'],
                          'solver': ['lbfgs']},
                         {'C': [100, 10, 1, 0.1, 0.01], 'penalty': ['l2'],
                          'solver': ['liblinear']},
                         {'C': [100, 10, 1, 0.1, 0.01], 'penalty': ['l2'],
                          'solver': ['sag']},
                         {'C': [100, 10, 1, 0.1, 0.01], 'penalty': ['l2'],
                          'solver': ['saga']},
                         {'C': [100, 10, 1, 0.1, 0.01], 'l1_ratio': [0.5],
                          'penalty': ['elasticnet'], 'solver': ['saga']}],
             scoring='accuracy')

In [25]:
grid.best_params_

{'C': 0.01, 'penalty': 'l2', 'solver': 'newton-cg'}

In [26]:
grid.best_score_

0.8785714285714287

In [28]:
y_pred = grid.predict(X_test)

In [29]:
score = accuracy_score(y_test,y_pred)
print(score)
cm = confusion_matrix(y_test,y_pred)
print(cm)
print(classification_report(y_test,y_pred))

0.8533333333333334
[[124  11]
 [ 33 132]]
              precision    recall  f1-score   support

           0       0.79      0.92      0.85       135
           1       0.92      0.80      0.86       165

    accuracy                           0.85       300
   macro avg       0.86      0.86      0.85       300
weighted avg       0.86      0.85      0.85       300



In [30]:
## Randomised SearchCV


## Randomised SearchCV


In [32]:
from sklearn.model_selection import RandomizedSearchCV

In [33]:
model = LogisticRegression()
randomcv = RandomizedSearchCV(estimator=model,param_distributions=params,cv=5,scoring='accuracy')

In [34]:
randomcv.fit(X_train,y_train)

c:\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Python312\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


RandomizedSearchCV(cv=5, estimator=LogisticRegression(),
                   param_distributions=[{'C': [100, 10, 1, 0.1, 0.01],
                                         'penalty': ['l1'],
                                         'solver': ['liblinear']},
                                        {'C': [100, 10, 1, 0.1, 0.01],
                                         'penalty': ['l1'],
                                         'solver': ['saga']},
                                        {'C': [100, 10, 1, 0.1, 0.01],
                                         'penalty': ['l2'],
                                         'solver': ['newton-cg']},
                                        {'C': [100, 10, 1, 0.1, 0.01],
                                         'penalty': ['l2'],
                                         'solver': ['lbfgs']},
                                        {'C': [100, 10, 1, 0.1, 0.01],
                                         'penalty': ['l2'],
                                         'solver': ['liblinear']},
                                        {'C': [100, 10, 1, 0.1, 0.01],
                                         'penalty': ['l2'], 'solver': ['sag']},
                                        {'C': [100, 10, 1, 0.1, 0.01],
                                         'penalty': ['l2'],
                                         'solver': ['saga']},
                                        {'C': [100, 10, 1, 0.1, 0.01],
                                         'l1_ratio': [0.5],
                                         'penalty': ['elasticnet'],
                                         'solver': ['saga']}],
                   scoring='accuracy')

In [35]:
randomcv.best_score_

0.8785714285714287

In [36]:
randomcv.best_params_

{'solver': 'saga', 'penalty': 'l2', 'C': 0.01}

In [38]:
y_pred=randomcv.predict(X_test)
y_pred

array([0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0,
       0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1,
       0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1,
       1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1,
       1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1,
       0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1,
       1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0])

In [39]:
score = accuracy_score(y_test,y_pred)
print(score)
cm = confusion_matrix(y_test,y_pred)
print(cm)
print(classification_report(y_test,y_pred))

0.8533333333333334
[[124  11]
 [ 33 132]]
              precision    recall  f1-score   support

           0       0.79      0.92      0.85       135
           1       0.92      0.80      0.86       165

    accuracy                           0.85       300
   macro avg       0.86      0.86      0.85       300
weighted avg       0.86      0.85      0.85       300



## Logisic Regression for multiclass classification problem

In [40]:
## create the dataset
X,y= make_classification(n_samples=1000,n_features=10,n_informative=3,n_classes=3,random_state=42)

In [41]:
# train test split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.30,random_state=42)

In [43]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
logistic = OneVsRestClassifier(LogisticRegression())
logistic.fit(X_train,y_train)
y_pred = logistic.predict(X_test)

In [44]:
y_pred

array([2, 0, 1, 2, 0, 1, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 1, 0, 0, 0, 0,
       2, 0, 1, 0, 2, 1, 0, 2, 2, 0, 1, 0, 0, 2, 2, 2, 1, 0, 0, 1, 2, 0,
       1, 2, 1, 0, 1, 1, 2, 0, 1, 0, 2, 2, 2, 2, 1, 2, 0, 2, 2, 2, 1, 1,
       0, 1, 0, 1, 0, 2, 2, 0, 0, 0, 2, 1, 1, 2, 2, 0, 2, 1, 0, 1, 1, 2,
       1, 1, 2, 2, 1, 2, 2, 2, 1, 0, 0, 0, 0, 2, 1, 0, 2, 1, 1, 0, 0, 2,
       0, 1, 2, 0, 0, 0, 1, 1, 2, 2, 1, 0, 0, 1, 2, 0, 0, 1, 0, 2, 1, 0,
       2, 2, 2, 0, 0, 1, 2, 1, 2, 1, 2, 1, 2, 2, 2, 0, 0, 1, 2, 0, 0, 2,
       1, 0, 0, 0, 2, 0, 0, 0, 2, 2, 2, 2, 2, 1, 2, 1, 0, 2, 1, 0, 0, 2,
       0, 1, 1, 2, 2, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 2, 1, 2, 2, 1, 2,
       0, 1, 2, 2, 1, 1, 0, 2, 1, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 2, 2, 2, 0, 0, 0, 0, 0, 2, 2, 0, 0, 2, 2, 1, 0, 1, 2, 0, 0, 2,
       0, 2, 0, 0, 2, 2, 0, 2, 2, 0, 1, 2, 2, 0, 1, 1, 0, 1, 2, 0, 2, 2,
       0, 0, 0, 2, 1, 2, 0, 2, 2, 2, 0, 2, 0, 2, 2, 0, 0, 1, 0, 2, 2, 2,
       0, 2, 2, 0, 2, 2, 0, 2, 0, 0, 2, 2, 0, 1])

In [45]:
score = accuracy_score(y_test,y_pred)
print(score)
cm = confusion_matrix(y_test,y_pred)
print(cm)
print(classification_report(y_test,y_pred))

0.68
[[79 16  8]
 [31 38 27]
 [ 3 11 87]]
              precision    recall  f1-score   support

           0       0.70      0.77      0.73       103
           1       0.58      0.40      0.47        96
           2       0.71      0.86      0.78       101

    accuracy                           0.68       300
   macro avg       0.67      0.67      0.66       300
weighted avg       0.67      0.68      0.66       300



### Hyperparameter tuning


In [50]:
from scipy.stats import uniform
params = {
    'estimator__C': uniform(0.1, 3.0),
    'estimator__solver': ['lbfgs', 'liblinear', 'newton-cg', 'sag', 'saga'],
    'estimator__max_iter': [100, 200, 300, 400, 500],
    'estimator__penalty': ['l1', 'l2'],
    'estimator__tol': uniform(1e-5, 1e-3)
}

In [51]:
#Init the model
# Create base logistic regression model
base_estimator = LogisticRegression()
# Create OneVsRestClassifier with the base estimator
ovr = OneVsRestClassifier(estimator=base_estimator)

# Set up RandomizedSearchCV
random_search = RandomizedSearchCV(
    ovr,
    param_distributions=params,
    n_iter=10,
    cv=5,
    random_state=42,
    scoring='accuracy',
    n_jobs=-1
)

In [52]:
#Set up randomCV
# Set up RandomizedSearchCV
random_search = RandomizedSearchCV(
    ovr,
    param_distributions=params,
    n_iter=10,
    cv=5,
    random_state=42,
    scoring='accuracy',
    n_jobs=-1
)
# Fit the model
random_search.fit(X_train, y_train)


c:\Python312\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
10 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Python312\Lib\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Python312\Lib\site-packages\sklearn\multiclass.py", line 370, in fit
    self.estimators_ = Parallel(n_jobs=self.n_jobs, verbose=self.verbose)(


RandomizedSearchCV(cv=5,
                   estimator=OneVsRestClassifier(estimator=LogisticRegression()),
                   n_jobs=-1,
                   param_distributions={'estimator__C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000019702DBB320>,
                                        'estimator__max_iter': [100, 200, 300,
                                                                400, 500],
                                        'estimator__penalty': ['l1', 'l2'],
                                        'estimator__solver': ['lbfgs',
                                                              'liblinear',
                                                              'newton-cg',
                                                              'sag', 'saga'],
                                        'estimator__tol': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000019704FE9DF0>},
                   random_state=42, scoring='accuracy')

In [53]:


# Print best parameters
print("Best parameters:", random_search.best_params_)

# Get best estimator
best_model = random_search.best_estimator_

# Print best score
print("\nBest cross-validation score:", random_search.best_score_)

Best parameters: {'estimator__C': 0.23935123815999318, 'estimator__max_iter': 300, 'estimator__penalty': 'l1', 'estimator__solver': 'saga', 'estimator__tol': 0.000460499251969543}

Best cross-validation score: 0.6271428571428571


In [54]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from scipy.stats import uniform, loguniform

# Create base logistic regression model with safe defaults
base_estimator = LogisticRegression(max_iter=1000)  # Increased max_iter for better convergence

# Create OneVsRestClassifier
ovr = OneVsRestClassifier(estimator=base_estimator)

# Define parameter distributions with compatible combinations
params = {
    'estimator__C': loguniform(1e-3, 1e3),  # Log-uniform distribution for better C value exploration
    'estimator__solver': ['lbfgs', 'saga'],  # Reduced to most reliable solvers
    'estimator__max_iter': [1000, 2000, 3000],  # Increased iterations
    'estimator__tol': loguniform(1e-5, 1e-3),
    'estimator__penalty': ['l2']  # Stick with l2 penalty for stability
}

# Set up RandomizedSearchCV with error handling
random_search = RandomizedSearchCV(
    ovr,
    param_distributions=params,
    n_iter=10,
    cv=5,
    random_state=42,
    scoring='accuracy',
    n_jobs=-1,
    error_score='raise',  # This will help identify specific issues
    verbose=2  # Added verbosity to see progress
)

# Fit the model with error handling
try:
    random_search.fit(X_train, y_train)
    
    # Print results
    print("\nBest parameters:", random_search.best_params_)
    print("\nBest cross-validation score:", random_search.best_score_)
    print("\nAll CV results:", random_search.cv_results_['mean_test_score'])
    
    # Get best estimator
    best_model = random_search.best_estimator_
    
except Exception as e:
    print(f"An error occurred during fitting: {str(e)}")
    # Additional debug information
    print("\nShape of X_train:", X_train.shape)
    print("Shape of y_train:", y_train.shape)
    print("Unique values in y_train:", np.unique(y_train))

Fitting 5 folds for each of 10 candidates, totalling 50 fits

Best parameters: {'estimator__C': 0.002231010801867922, 'estimator__max_iter': 1000, 'estimator__penalty': 'l2', 'estimator__solver': 'saga', 'estimator__tol': 1.9307837536547128e-05}

Best cross-validation score: 0.6428571428571428

All CV results: [0.62285714 0.62142857 0.64285714 0.62142857 0.62142857 0.62142857
 0.64       0.64       0.62142857 0.64      ]


In [55]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from scipy.stats import loguniform
from sklearn.metrics import classification_report
import numpy as np

# First, let's check class distribution
print("Class distribution:")
print(np.bincount(y_train))

# Create a more sophisticated pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Scale features
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),  # Add polynomial features
    ('classifier', OneVsRestClassifier(LogisticRegression(max_iter=2000)))
])

# Define a broader parameter grid
params = {
    'poly__degree': [1, 2],  # Try with and without polynomial features
    'classifier__estimator__C': loguniform(1e-4, 1e4),
    'classifier__estimator__solver': ['saga', 'lbfgs'],
    'classifier__estimator__penalty': ['l2'],
    'classifier__estimator__class_weight': ['balanced', None],  # Handle class imbalance
    'classifier__estimator__tol': loguniform(1e-6, 1e-3),
    'classifier__estimator__max_iter': [2000],
    'classifier__estimator__warm_start': [True, False]
}

# Set up RandomizedSearchCV with more iterations
random_search = RandomizedSearchCV(
    pipeline,
    param_distributions=params,
    n_iter=20,  # Increased number of iterations
    cv=5,
    scoring=['accuracy', 'f1_weighted', 'precision_weighted', 'recall_weighted'],
    refit='f1_weighted',  # Use F1 score for selecting best model
    n_jobs=-1,
    verbose=2,
    random_state=42
)

# Fit the model
random_search.fit(X_train, y_train)

# Print detailed results
print("\nBest parameters:", random_search.best_params_)
print("\nBest score (F1-weighted):", random_search.best_score_)

# Get best model
best_model = random_search.best_estimator_

# Make predictions
y_pred = best_model.predict(X_test)

# Print detailed classification report
print("\nDetailed Classification Report:")
print(classification_report(y_test, y_pred))

# Feature importance analysis (for original features)
def get_feature_importance(model, feature_names):
    if hasattr(model.named_steps['classifier'].estimator_, 'coef_'):
        # Get coefficients for original features (before polynomial transformation)
        coefficients = model.named_steps['classifier'].estimator_.coef_
        importance = np.abs(coefficients).mean(axis=0)
        if model.named_steps['poly'].degree == 1:
            feature_importance = list(zip(feature_names, importance))
        else:
            # If using polynomial features, just show original feature importance
            feature_importance = list(zip(feature_names, importance[:len(feature_names)]))
        return sorted(feature_importance, key=lambda x: abs(x[1]), reverse=True)
    return None

# If you have feature names, analyze their importance
if 'feature_names' in locals():
    print("\nFeature Importance:")
    importance = get_feature_importance(best_model, feature_names)
    if importance:
        for feat, imp in importance:
            print(f"{feat}: {imp:.4f}")

Class distribution:
[231 235 234]
Fitting 5 folds for each of 20 candidates, totalling 100 fits

Best parameters: {'classifier__estimator__C': 0.4452048365748854, 'classifier__estimator__class_weight': 'balanced', 'classifier__estimator__max_iter': 2000, 'classifier__estimator__penalty': 'l2', 'classifier__estimator__solver': 'lbfgs', 'classifier__estimator__tol': 3.972110727381911e-06, 'classifier__estimator__warm_start': True, 'poly__degree': 2}

Best score (F1-weighted): 0.8163463618646155

Detailed Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.85      0.86       103
           1       0.85      0.73      0.79        96
           2       0.76      0.88      0.82       101

    accuracy                           0.82       300
   macro avg       0.83      0.82      0.82       300
weighted avg       0.83      0.82      0.82       300

